## Advanced Lane Finding Project

In this project, the objective is to detect lane lines in images using Python and OpenCV. OpenCV means "Open-Source Computer Vision", which is a package that has many useful tools for analyzing images.

The goal and steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---


In [ ]:
# Get imports
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt

### Step 1: Compute the camera calibration using chessboard images

---

In [ ]:
# Define a helper function "objectAndImagePoints" to Calibrate camera using ChaseBoard

def objectAndImagePoints(image_dir):
    
    """ 
    This function takes in a set of images used for calibration,
    and outputs objpoints, imgpoints and corners to compute the 
    camera calibration and distortion coefficients using the cv2.calibrateCamera() function.

    input: images
    args: 
    output:objpoints, imgpoints, corners
    
    """
    
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)
    
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.
    
    # Make a list of calibration images
    images = image_dir
    
    # Step through the list and search for chessboard corners
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            cv2.drawChessboardCorners(img, (9,6), corners, ret)
            cv2.imshow('img', img)
            cv2.waitKey(500)

    cv2.destroyAllWindows()
    
    
    return objpoints, imgpoints, corners


In [ ]:
# Directory for calibration images
image_dir = glob.glob('camera_cal/calibration*.jpg')
# Get the objpoints, imgpoints, corners for use in distortion correction. 
# objpoints, imgpoints, corners = objectAndImagePoints(image_dir)

### Step 2: Distortion correction
---

In [ ]:
# Define a helper function "calibrate_undistort" to Calibrate camera using ChaseBoard
def calibrate_undistort(img):
    
    """
    This function takes image, objpoints, imgpoints as input, it then outputs undistorted image.
    It then uses the cv2.calibrateCamera() function on the inputs for distortion correction on
    the image using the cv2.undistort() function and obtains an outpur result.
    
    Input:img, objpoints, imgpoints
    args:
    Output: undistorted image
    
    """
    img = np.copy(img)
    img_size = (img.shape[1], img.shape[0])
    
    # Get the objpoints, imgpoints for use in distortion correction. 
    objpoints, imgpoints,_ = objectAndImagePoints(image_dir)
    
    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
    
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
    return dst


In [ ]:
def visualizeUndistortion(original_image, dist_image):
    # Visualize undistortion
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(original_image);
    ax1.set_title('Original Image', fontsize=30)

    ax2.imshow(dist_image);
    ax2.set_title('Undistorted Image', fontsize=30)
    

In [ ]:

def pipelineUndistort(img, destination_path, save_name):
    dst = calibrate_undistort(img)
    cv2.imwrite(destination_path + save_name, dst)
    
    undist_img = mpimg.imread(destination_path + save_name)
    
    return visualizeUndistortion(img, undist_img)
    

In [ ]:

# Files to undistort
img_cal = mpimg.imread('./camera_cal/calibration1.jpg')
img_straight_lines1 = mpimg.imread('./test_images/straight_lines1.jpg')
img_straight_lines2 = mpimg.imread('./test_images/straight_lines2.jpg')
img_test1 = mpimg.imread('./test_images/test1.jpg')
img_test2 = mpimg.imread('./test_images/test2.jpg')
img_test3 = mpimg.imread('./test_images/test3.jpg')
img_test4 = mpimg.imread('./test_images/test4.jpg')
img_test5 = mpimg.imread('./test_images/test5.jpg')
img_test6 = mpimg.imread('./test_images/test6.jpg')


In [ ]:
# Path to saved directoy 
dir_path = './output_images/'
save_name = ""


# Read undisorted files
undist_img_cal = pipelineUndistort(img_cal, dir_path, save_name='undistorted_cal.jpg')
undist_straight_lines1 = pipelineUndistort(img_straight_lines1, dir_path, save_name='undist_straight_lines1.jpg')
undist_straight_lines2 = pipelineUndistort(img_straight_lines2, dir_path, save_name='undist_straight_lines2.jpg')

undist_img_test1 = pipelineUndistort(img_test1,dir_path, save_name='undist_img_test1.jpg')
undist_img_test2 = pipelineUndistort(img_test2, dir_path, save_name='undist_img_test2.jpg')
undist_img_test3 = pipelineUndistort(img_test3,dir_path, save_name='undist_img_test3.jpg')
undist_img_test4 = pipelineUndistort(img_test4,dir_path, save_name='undist_img_test4.jpg')
undist_img_test5 = pipelineUndistort(img_test5,dir_path, save_name='undist_img_test5.jpg')
undist_img_test6 = pipelineUndistort(img_test6,dir_path, save_name='undist_img_test6.jpg')

### Step 3: Color and gradient threshold
---

In [ ]:
# Read image to use in color and gradient threshold
undistorted_straight_lines1 = mpimg.imread('./output_images/undist_straight_lines1.jpg')
plt.imshow(undistorted_straight_lines1);

In [ ]:
# Helper function
def pipelineColorAndGradientThreshold(image, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(image)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    
    
    # Combine the two binary thresholds
    binary_image = np.zeros_like(sxbinary)
    binary_image[(s_binary == 1) | (sxbinary == 1)] = 1
    return color_binary, binary_image
    
def visualizeColorTheshold(image1, image2):
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    f.tight_layout()

    ax1.imshow(image1)
    ax1.set_title('Color_binary', fontsize=15)

    ax2.imshow(image2, cmap='gray')
    ax2.set_title('Binary_image', fontsize=15)
    # plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
color_binary, binary_image = pipelineColorAndGradientThreshold(undistorted_straight_lines1)
visualizeColorTheshold(color_binary, binary_image)

### Step 4: Perspective transform
---

In [ ]:
# Helper functions
import perspective_transform_functions as ptf
import draw_perspective_lines as dpl

def visualizePerspectiveTransform(image1, image2):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    f.tight_layout()
    ax1.imshow(image1)
    ax1.set_title('Original Undistorted Image', fontsize=15)
    ax2.imshow(image2)
    ax2.set_title('Undistorted and Warped Image', fontsize=15)

In [ ]:
# Get region of interest source and destination 
src, dst = ptf.get_trapezoid(undistorted_straight_lines1)
src

In [ ]:
# Draw sraight lane lines on edge of the road
masked_image = dpl.region_of_interest(binary_image, src)
plt.imshow(masked_image);

In [ ]:
# Define the Hough transform parameters

rho = 2 # distance resolution in pixels of the Hough grid
theta = np.pi/180 # angular resolution in radians of the Hough grid
threshold = 10    # minimum number of votes (intersections in Hough grid cell)
min_line_len = 10 #minimum number of pixels making up a line
max_line_gap = 20    # maximum gap in pixels between connectable line segments


# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
line_img, lines = dpl.hough_lines(masked_image, rho, theta, threshold, min_line_len, max_line_gap)


In [ ]:
plt.figure(figsize=(12,8))
plt.imshow(line_img);

In [ ]:
continous_lines = dpl.getLeft_and_rightLane(undistorted_straight_lines1, lines)
final_image = dpl.drawSingleft_and_rightLane(undistorted_straight_lines1,continous_lines,thickness=6,)


In [ ]:

plt.figure(figsize=(12,8))
plt.imshow(final_image)
plt.show()

In [ ]:
# perspective transform on drawn image
# unwarped = ptf.get_original_perspective(img)
top_down = ptf.get_transformed_perspective(final_image)
visualizePerspectiveTransform(final_image, top_down)

In [ ]:
# perspective transform on lane lines. 
# unwarped = ptf.get_original_perspective(img)
top_down = ptf.get_transformed_perspective(binary_image)#undistorted_straight_lines1
birds_view = ptf.get_transformed_perspective(undistorted_straight_lines1)
visualizePerspectiveTransform(undistorted_straight_lines1, birds_view)

### Step 5: Detect lane pixels and fit to find the lane boundary

In [ ]:
histogram = np.sum(top_down[top_down.shape[0]//2:,:], axis=0)
plt.plot(histogram);

In [ ]:
# Helper functions
import detect_lane_pixels as detect_lane_pixels

# Visualize lane window
out_img,left_fit, right_fit = detect_lane_pixels.fit_polynomial(top_down)
# out_img,left_fit, right_fit = fit_polynomial(top_down)
plt.imshow(out_img);

In [ ]:
# Run image through the pipeline show lane pixels on road
result,left_fitx, right_fitx, ploty = detect_lane_pixels.search_around_poly(top_down)

# View your output
plt.imshow(result);

### Step 6: Determine the curvature of the lane and vehicle position with respect to center

In [ ]:
def measure_curvature_pixels(image):#binary_warped
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''
    
    result,left_fitx, right_fitx, ploty = detect_lane_pixels.search_around_poly(image)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad


# Calculate the radius of curvature in pixels for both lane lines
left_curverad, right_curverad = measure_curvature_pixels(top_down)

print(left_curverad, right_curverad)

In [ ]:
def measure_curvature_real(image):#binary_warped
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    
    image_center = image.shape[1] / 2
    #lane_width_px = 
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    result,left_fit_cr, right_fit_cr, ploty = detect_lane_pixels.search_around_poly(image)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    
    mean_curvature = np.int32(np.mean((left_curverad, right_curverad)))
    
    # Mean position of the lanes in the image
    leftx = left_fit_cr
    rightx = right_fit_cr
    lane_centerx = np.mean((leftx[-1], rightx[-1]))
    
    # Lane mean position relative to image center. 
    # If positive, the lane is offset to the left i.e. positive vehicle is to the right.
    vehicle_position_px = image_center - lane_centerx
    vehicle_position_cm = np.int32(vehicle_position_px * xm_per_pix * 100)
    
    
    return left_curverad, right_curverad #left_curverad, right_curverad


# # Calculate the radius of curvature in meters for both lane lines
left_curverad, right_curverad = measure_curvature_real(top_down)
print(left_curverad, 'm', right_curverad, 'm')


# left_curverad, right_curverad, vehicle_position_cm = measure_curvature_real(top_down)
# print(left_curverad, 'm', right_curverad, 'm', vehicle_position_cm)